# Developing Vertex AI pipelines with the TFX SDK



In [1]:
import os
import sys
import kfp
import tensorflow as tf
import tfx

from kfp.v2.google.client import AIPlatformClient

from tfx.dsl.components.base import executor_spec
from tfx.components.trainer import executor as trainer_executor
from tfx.extensions.google_cloud_ai_platform.trainer import executor as ai_platform_trainer_executor
from tfx.orchestration import data_types
from tfx.orchestration.kubeflow.v2 import kubeflow_v2_dag_runner
from tfx.orchestration.local.local_dag_runner import LocalDagRunner
from tfx.orchestration.metadata import sqlite_metadata_connection_config
from tfx.proto import trainer_pb2


In [2]:
tfx.__version__

'1.0.0-rc1'

In [3]:
%load_ext autoreload
%autoreload 2

## Configure lab settings

In [4]:
PROJECT = 'jk-vertexai-ws'
REGION = 'us-central1'
PREFIX = 'jk'

STAGING_BUCKET = f'gs://{PREFIX}-bucket'
VERTEX_SA = f'training-sa@{PROJECT}.iam.gserviceaccount.com'
PIPELINES_SA = f'pipelines-sa@{PROJECT}.iam.gserviceaccount.com'

DATA_ROOT_URI = 'gs://workshop-datasets/covertype/small'
SCHEMA_FOLDER = f'{STAGING_BUCKET}/tfx/schema'
TFX_IMAGE = f'gcr.io/{PROJECT}/tfxcovertype'

In [5]:
!gsutil cp pipeline/schema/schema.pbtxt {SCHEMA_FOLDER}/schema.pbtxt

Copying file://pipeline/schema/schema.pbtxt [Content-Type=application/octet-stream]...
/ [1 files][  2.9 KiB/  2.9 KiB]                                                
Operation completed over 1 objects/2.9 KiB.                                      


## Review the pipeline design

## Compile the pipeline

In [7]:
from pipeline import pipeline

pipeline_name = 'covertype-continuous-training'

pipeline_def = pipeline.create_pipeline(
    pipeline_name=pipeline_name,
    pipeline_root=f'{STAGING_BUCKET}/pipelines/{pipeline_name}',
    serving_model_uri=f'{STAGING_BUCKET}/models/Covertype',
    data_root_uri=data_types.RuntimeParameter(
        name='data_root_uri',
        ptype=str,
        default=DATA_ROOT_URI
    ),
    schema_folder_uri=SCHEMA_FOLDER,
    train_steps = 1000,
    eval_steps = 500,
)

In [9]:
pipeline_definition_file = 'covertype.json'

runner = kubeflow_v2_dag_runner.KubeflowV2DagRunner(
    config=kubeflow_v2_dag_runner.KubeflowV2DagRunnerConfig(
        default_image=TFX_IMAGE
    ),
    output_filename=pipeline_definition_file,
)

pipeline_json = runner.run(pipeline_def)

## Prepare a TFX container

In [10]:
!gcloud builds submit --tag {TFX_IMAGE} pipeline

Creating temporary tarball archive of 11 file(s) totalling 42.7 KiB before compression.
Uploading tarball of [pipeline] to [gs://jk-vertexai-ws_cloudbuild/source/1623805684.827227-cc72b60009ee4c18944ebb4cf1906a90.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/jk-vertexai-ws/locations/global/builds/b7aab361-3f5d-49ed-bd13-5b3df415c013].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/b7aab361-3f5d-49ed-bd13-5b3df415c013?project=1058712423388].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "b7aab361-3f5d-49ed-bd13-5b3df415c013"

FETCHSOURCE
Fetching storage object: gs://jk-vertexai-ws_cloudbuild/source/1623805684.827227-cc72b60009ee4c18944ebb4cf1906a90.tgz#1623805685244919
Copying gs://jk-vertexai-ws_cloudbuild/source/1623805684.827227-cc72b60009ee4c18944ebb4cf1906a90.tgz#1623805685244919...
/ [1 files][ 11.6 KiB/ 11.6 KiB]                                                
Operation completed over 1 o

## Submitting pipeline runs

In [11]:
pipeline_client = AIPlatformClient(
    project_id=PROJECT,
    region=REGION,
)

In [12]:
parameter_values = {
    'data_root_uri': DATA_ROOT_URI,
}

pipeline_client.create_run_from_job_spec(
    job_spec_path=pipeline_definition_file,
    parameter_values=parameter_values,
    enable_caching=False,
    service_account=PIPELINES_SA
)

{'name': 'projects/1058712423388/locations/us-central1/pipelineJobs/covertype-continuous-training-20210616011701',
 'displayName': 'covertype-continuous-training-20210616011701',
 'createTime': '2021-06-16T01:17:01.274419Z',
 'updateTime': '2021-06-16T01:17:01.274419Z',
 'pipelineSpec': {'deploymentConfig': {'@type': 'type.googleapis.com/ml_pipelines.PipelineDeploymentConfig',
   'executors': {'DataTransformer_executor': {'container': {'image': 'gcr.io/jk-vertexai-ws/tfxcovertype',
      'command': ['python',
       '-m',
       'tfx.orchestration.kubeflow.v2.container.kubeflow_v2_run_executor'],
      'args': ['--executor_class_path',
       'tfx.components.transform.executor.Executor',
       '--json_serialized_invocation_args',
       '{{$}}']}},
    'ExampleValidator_executor': {'container': {'image': 'gcr.io/jk-vertexai-ws/tfxcovertype',
      'command': ['python',
       '-m',
       'tfx.orchestration.kubeflow.v2.container.kubeflow_v2_run_executor'],
      'args': ['--executor_c

## Extracting pipeline run metadata

In [1]:
from google.cloud import aiplatform

In [3]:
pipeline_name = 'covertype-continuous-training'


pipeline_df = aiplatform.get_pipeline_df(pipeline=pipeline_name)
pipeline_df

,pipeline_name,run_name,param.input:data_root_uri,param.input:schema_folder_uri
0,covertype-continuous-training,covertype-continuous-training-20210616014601,gs://workshop-datasets/covertype/small,NaN
1,covertype-continuous-training,covertype-continuous-training-20210616011701,gs://workshop-datasets/covertype/small,NaN
2,covertype-continuous-training,covertype-continuous-training-20210616000943,gs://workshop-datasets/covertype/small,NaN
3,covertype-continuous-training,covertype-continuous-training-20210615235033,gs://workshop-datasets/covertype/small,NaN
4,covertype-continuous-training,covertype-continuous-training-20210615230519,gs://workshop-datasets/covertype/small,NaN
5,covertype-continuous-training,covertype-continuous-training-20210615224934,gs://workshop-datasets/covertype/small,NaN
6,covertype-continuous-training,covertype-continuous-training-20210615224805,gs://workshop-datasets/covertype/small,NaN
7,covertype-continuous-training,covertype-continuous-training-20210615222242,gs://workshop-datasets/covertype/small,NaN
8,covertype-continuous-training,covertype-continuous-training-20210615221433,gs://workshop-datasets/covertype/small,gs://jk-bucket/tfx/schema
